<a href="https://colab.research.google.com/github/nirb28/llm/blob/main/code-conversion/TestModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running in Colab!")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
else:
    print("Not running in Colab.")

def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/nn_catalyst/' + relativePath
    else:
        from utils import get_project_root
        return get_project_root() + "/" + relativePath

from huggingface_hub import login
import os
if IN_COLAB:
    from google.colab import userdata
    HF_TOKEN=userdata.get('HF_TOKEN')
else:
    from dotenv import load_dotenv
    load_dotenv()
    HF_TOKEN=os.getenv("HF_TOKEN")

login(token = HF_TOKEN)

Running in Colab!
Mounted at /content/drive
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoderbase")
tokenizer.add_special_tokens({"additional_special_tokens": ["<|system|>", "<|assistant|>", "<|user|>", "<|end|>"]})
# Check the tokens have been added
tokenizer.special_tokens_map

{
    "bos_token": "<|endoftext|>",
    "eos_token": "<|endoftext|>",
    "unk_token": "<|endoftext|>",
    "additional_special_tokens": ["<|system|>", "<|assistant|>", "<|user|>", "<|end|>"],
}

tokenizer_config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'additional_special_tokens': ['<|system|>',
  '<|assistant|>',
  '<|user|>',
  '<|end|>']}

In [3]:
print("Authentication successful and AutoTokenizer loaded for StarCoder base.")
print("Tokenizer vocabulary size:", len(tokenizer))


Authentication successful and AutoTokenizer loaded for StarCoder base.
Tokenizer vocabulary size: 49156


In [4]:

# Example usage
text = "def hello_world():"
encoded = tokenizer.encode(text)
print(f"Encoded '{text}': {encoded}")
decoded = tokenizer.decode(encoded)
print(f"Decoded: {decoded}")

Encoded 'def hello_world():': [589, 17964, 81, 5860, 2262]
Decoded: def hello_world():


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# Load the StarcoderBase model and tokenizer
model_name = "bigcode/starcoderbase"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto",
    offload_folder="offload", offload_state_dict = True,
    torch_dtype=torch.float32
)

def convert_sas_to_python(sas_code):
    # Tokenize the SAS code
    inputs = tokenizer(sas_code, return_tensors="pt")

    # Generate Python code
    outputs = model.generate(**inputs, max_length=1024)
    python_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return python_code


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 9854.25 MB. The target location /root/.cache/huggingface/hub/models--bigcode--starcoderbase/blobs only has 0.00 MB free disk space.
  warnings.warn(


pytorch_model-00005-of-00007.bin:  94%|#########3| 9.21G/9.85G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

In [ ]:

# Example usage
sas_code = """
data sample;
  set old_data;
  where age > 21;
run;
"""
python_code = convert_sas_to_python(sas_code)
print("Converted Python Code:\n", python_code)